In [35]:
import numpy as np
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic
from tqdm import trange
from numba import njit

%load_ext snakeviz
# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [36]:
def compute_RMS_error(D1, D2):
    e_rms = np.sqrt(np.sum(D1**2 + D2**2) / (2*np.size(D1, 0)))
    return e_rms

In [37]:
def plot_lines_points_and_image(img_pts, img, l, path, save=False):

    fig = plt.figure(figsize=(9,6))
    ax = plt.axes()
    
    cv.compute_and_plot_lines(l, img, ax)
    ax.plot(img_pts[0], img_pts[1], 'o', color='blue', label='Random points')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    # plt.gca().invert_yaxis()
    # ax.invert_yaxis()
    ax.set_aspect('equal')
    ax.legend(loc="upper right")
    # ax.margins(x=0.1, y=0.1) 
    fig.tight_layout()

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if save:
        fig.savefig(path, dpi=300)
    plt.show()


In [38]:
def plot_histogram(data, path, mean, save=False):
    fig = plt.figure()
    plt.hist(data, bins=100, color='tab:blue')
    plt.axvline(mean, linestyle='--', color='red', label='Mean: {}'.format(round(mean, 2)))
    plt.xlabel('Error')
    plt.ylabel('Frequency')
    # plt.xlim([0,70])
    plt.legend(loc="upper right")
    fig.tight_layout()
    if save:
        fig.savefig(path, dpi=300)
    plt.show()

In [39]:
def compute_point_line_distance_2D(l, p):
    # a = l[0,:]
    # b = l[1,:]
    # c = l[2,:]

    # x = p[0,:]
    # y = p[1,:]

    # D = np.abs(a*x + b*y + c) / (a**2 + b**2)**0.5

    numerator = np.abs(np.einsum("ij, ij->j", p, l))
    denominator = np.linalg.norm(l[:-1], axis=0)
    D = numerator / denominator

    return D

def compute_epipolar_lines(F, x1, x2):
    # l3 = F @ x1
    # l4 = F.T @ x2

    l2 = np.einsum("ij,jk->ik", F, x1)
    l1 = np.einsum("ji,jk->ik", F, x2)
    return l1, l2

def compute_epipolar_errors(F, x1, x2):
    l1, l2 = compute_epipolar_lines(F, x1, x2)
    D1 = compute_point_line_distance_2D(l1, x1)
    D2 = compute_point_line_distance_2D(l2, x2)
    return D1, D2

In [47]:
# @njit()
def estimate_E_robust(K, x1_norm, x2_norm):
    
    err_threshold_px = 2
    err_threshold = err_threshold_px / K[0,0]
    
    # alpha = 0.95
    # E = cv.estimate_E_DLT(x1_norm, x2_norm, enforce=True, verbose=False)
    # D1, D2, _ = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
    # inliers = ((D1**2 + D2**2) / 2) < err_threshold**2
    # epsilon = np.sum(inliers) / np.size(inliers)
    # s = 8
    # T = np.ceil(np.log(1-alpha) / np.log(1-epsilon**s))

    s = 8

    best_inliers = None
    best_E = None
    max_inliers = 0

    for t in trange(10000):

        rand_mask = np.random.choice(np.size(x1_norm,1), s, replace=False)
        E = cv.estimate_E_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], enforce=True, verbose=False)

        D1, D2 = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
        inliers = ((D1**2 + D2**2) / 2) < err_threshold**2

        n_inliers = np.sum(inliers)

        if n_inliers > max_inliers:
            best_inliers = np.copy(inliers)
            best_E = np.copy(E)
            max_inliers = n_inliers

            print(np.sum(inliers), end='\r')
        
    return best_E, best_inliers

In [41]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-4'
data = r'\data-2023\data'
compex = r'\compEx1data.mat'
img1 = r'\round_church1.jpg'
img2 = r'\round_church2.jpg'
report = r'\report-images'

round_church1 = cv.load_image(path+data+img1)
round_church2 = cv.load_image(path+data+img2)
K = cv.convert_mat_to_np(path+data+compex, 'K')
K_inv = np.linalg.inv(K)
x = cv.convert_mat_to_np(path+data+compex, 'x')
x1 = cv.dehomogenize(x[0,0])
x2 = cv.dehomogenize(x[0,1])

ransac = True
plt_pts = False
plt_hist = False
save = False
snakeviz = False

x1_norm = cv.transform_and_dehomogenize(K_inv, x1)
x2_norm = cv.transform_and_dehomogenize(K_inv, x2)

In [48]:
if ransac:
    i = 2 # NEVER 1
    E, inliers = estimate_E_robust(K, x1_norm, x2_norm)
    # np.save(path+data+'/CE1_E_robust_{}.npy'.format(i), E)
    # np.save(path+data+'/CE1_inliers_{}.npy'.format(i), inliers)
    print('No. inliers:', np.sum(inliers))
elif snakeviz:
    %snakeviz estimate_E_robust(K, x1_norm, x2_norm)
else:
    E = cv.estimate_E_DLT(x1_norm, x2_norm, enforce=True, verbose=False)

  0%|          | 20/10000 [00:00<00:57, 172.48it/s]

  1%|          | 54/10000 [00:00<01:27, 113.66it/s]

  4%|▎         | 373/10000 [00:03<01:06, 144.80it/s]

  6%|▋         | 635/10000 [00:05<01:05, 142.60it/s]

 26%|██▋       | 2633/10000 [00:12<00:17, 412.65it/s]

 30%|███       | 3006/10000 [00:13<00:16, 416.15it/s]

 43%|████▎     | 4312/10000 [00:16<00:13, 417.34it/s]

 52%|█████▏    | 5196/10000 [00:18<00:09, 481.21it/s]

100%|██████████| 10000/10000 [00:31<00:00, 312.59it/s]

No. inliers: 5561


In [32]:
i = 1
if ransac:
    E = np.load(path+data+'/CE1_E_robust_{}.npy'.format(i))
inliers = np.load(path+data+'/CE1_inliers_{}.npy'.format(i))
F = cv.convert_E_to_F(E, K, K)

print('\nE:', E)
print('F:', F)



# All points
D1, D2 = cv.compute_epipolar_errors(F, x1, x2)
e_rms = compute_RMS_error(D1, D2)

print('\nAll points:')
print('Ransac:', ransac)
print('Mean distance 1:', np.mean(D1))
print('Mean distance 2:', np.mean(D2))
print('RMS error:', e_rms)

path1 = path+report+'/CE1_hist1_ransac={}_all_points_{}.png'.format(ransac, i)
path2 = path+report+'/CE1_hist2_ransac={}_all_points_{}.png'.format(ransac, i)

if plt_hist:
    plot_histogram(D1, path1, np.mean(D1), save)
    plot_histogram(D2, path2, np.mean(D2), save)



# Inliers
D1, D2 = cv.compute_epipolar_errors(F, x1[:,inliers], x2[:,inliers])
e_rms = compute_RMS_error(D1, D2)

print('\nInliers:')
print('Ransac:', ransac)
print('Mean distance 1:', np.mean(D1))
print('Mean distance 2:', np.mean(D2))
print('RMS error:', e_rms)
print('No. inliers:', np.sum(inliers))

path1 = path+report+'/CE1_hist1_ransac={}_inliers_{}.png'.format(ransac, i)
path2 = path+report+'/CE1_hist2_ransac={}_inliers{}.png'.format(ransac, i)

if plt_hist:
    plot_histogram(D1, path1, np.mean(D1), save)
    plot_histogram(D2, path2, np.mean(D2), save)



# Plot
if ransac:
    x1_norm = x1_norm[:,inliers]
    x2_norm = x2_norm[:,inliers]
    x1 = x1[:,inliers]
    x2 = x2[:,inliers]

rand_mask = np.random.choice(np.size(x2_norm,1), 20, replace=False)
x1_rand = x1[:,rand_mask]
x2_rand = x2[:,rand_mask]
l1, l2 = cv.compute_epipolar_lines(F, x1_rand, x2_rand)

path1 = path+report+'/CE1_round_church1_ransac={}_{}.png'.format(ransac, i)
path2 = path+report+'/CE1_round_church2_ransac={}_{}.png'.format(ransac, i)

if plt_pts:
    plot_lines_points_and_image(x1_rand, round_church1, l1, path1, save)
    plot_lines_points_and_image(x2_rand, round_church2, l2, path2, save)


E: [[ -0.08611505  49.90346579  -8.45855958]
 [-54.18922505  -1.58372458  28.12759202]
 [  9.67883597 -35.07176971   1.        ]]
F: [[-8.41836645e-09  4.86994758e-06 -5.03129184e-03]
 [-5.28818352e-06 -1.54282994e-07  1.15986920e-02]
 [ 5.59532959e-03 -1.26371702e-02  1.00000000e+00]]

All points:
Ransac: True
Mean distance 1: 58.218871126293756
Mean distance 2: 71.45505427055761
RMS error: 211.08275098532155

Inliers:
Ransac: True
Mean distance 1: 0.2689665486285598
Mean distance 2: 0.30710082051218507
RMS error: 0.377877330889468
No. inliers: 5808


In [ ]:
# All points:
# Ransac: False
# Mean distance 1: 61.33360616690827
# Mean distance 2: 63.46892741628846
# RMS error: 155.95832290161417

# Inliers:
# Ransac: False
# Mean distance 1: 16.405076850386244
# Mean distance 2: 17.255170603651873
# RMS error: 24.840289353844906
# No. inliers: 5808

# All points:
# Ransac: True
# Mean distance 1: 58.218871126293756
# Mean distance 2: 71.45505427055761
# RMS error: 211.08275098532155

# Inliers:
# Ransac: True
# Mean distance 1: 0.2689665486285598
# Mean distance 2: 0.30710082051218507
# RMS error: 0.377877330889468
# No. inliers: 5808